In [1]:
#Load libraries
import os
import pandas as pd
import numpy as np
#from fancyimpute import KNN   
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import sklearn

In [2]:
os.chdir("C:/Users/PrasadaRao/Desktop/Cab rental")
os.getcwd()

'C:\\Users\\PrasadaRao\\Desktop\\Cab rental'

In [3]:
train_data = pd.read_csv("train_cab.csv")
train_data.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1.0
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1.0
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2.0
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1.0
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1.0


In [4]:
test_data = pd.read_csv("test.csv")
test_data.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [5]:
train_data.isnull().sum()

fare_amount          24
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count      55
dtype: int64

In [6]:
# removing nulls as there are only 55 null out of 16000 records which is aproximately 0.3 % of the whole data 
train_data.dropna(how='any',inplace=True)

In [7]:
print(train_data.isnull().sum())
print(train_data.shape)

fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64
(15988, 7)


In [8]:
### Removing pickup date time column 

train_data.drop(['pickup_datetime'],axis=1,inplace=True)

In [10]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 6 columns):
pickup_datetime      9914 non-null object
pickup_longitude     9914 non-null float64
pickup_latitude      9914 non-null float64
dropoff_longitude    9914 non-null float64
dropoff_latitude     9914 non-null float64
passenger_count      9914 non-null int64
dtypes: float64(4), int64(1), object(1)
memory usage: 464.8+ KB


In [11]:
train_data.dtypes

fare_amount           object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      float64
dtype: object

In [12]:
test_data.isna().sum()

pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [13]:
train_data['fare_amount'].value_counts()

6.5      757
4.5      669
8.5      627
5.3      473
4.9      462
        ... 
77.7       1
31.2       1
21.54      1
59.5       1
10.7       1
Name: fare_amount, Length: 467, dtype: int64

In [14]:
# treat values in fare amount 
train_data['fare_amount']= train_data['fare_amount'].replace("430-","430")

In [15]:
train_data['fare_amount']= train_data['fare_amount'].apply(pd.to_numeric)

In [16]:
train_data.loc[train_data['fare_amount'] < 0, 'fare_amount']

2039    -2.9
2486    -2.5
13032   -3.0
Name: fare_amount, dtype: float64

In [17]:
train_data = train_data[(train_data['fare_amount'] > 0)]
train_data.reset_index(drop=True,inplace=True)

In [18]:
train_data.shape

(15984, 6)

In [19]:
# check for outliers in passenger count 
# assumption is that a cab cannot carry more than 6 passengers at a time 
train_data.loc[train_data['passenger_count'] > 6,'passenger_count']

217      236.0
246      456.0
276     5334.0
337      535.0
367      354.0
923      554.0
959       53.0
995       35.0
1059     345.0
1098    5345.0
1152     536.0
1194      43.0
8326      53.0
8365      58.0
8426     537.0
8491      87.0
8551      43.0
8635     531.2
8905     557.0
Name: passenger_count, dtype: float64

In [20]:
train_data = train_data[(train_data['passenger_count'] < 7)]
train_data.reset_index(drop=True,inplace=True)

In [21]:
print(train_data.shape)

(15965, 6)


In [22]:
# Eliminating rows which has same pickup and drop latitude 

train_data.loc[(train_data['pickup_longitude'] == train_data['dropoff_longitude'])]

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
10,5.5,0.000000,0.000000,0.000000,0.000000,3.0
14,5.0,0.000000,0.000000,0.000000,0.000000,1.0
98,52.0,-74.035839,40.747320,-74.035839,40.747320,1.0
116,8.0,0.000000,0.000000,0.000000,0.000000,2.0
178,6.5,-73.998488,40.726303,-73.998488,40.726303,1.0
...,...,...,...,...,...,...
15828,10.9,0.000000,0.000000,0.000000,0.000000,5.0
15836,3.3,0.000000,0.000000,0.000000,0.000000,3.0
15913,10.9,0.000000,0.000000,0.000000,0.000000,2.0
15921,5.5,-73.984123,40.760387,-73.984123,40.760387,6.0


In [23]:
train_data = train_data.loc[~(train_data['pickup_longitude'] == train_data['dropoff_longitude'])]
print(train_data.shape)

(15507, 6)


In [24]:
#Even if the latitude is same, the longitude is different. Hence, we are not removing these rows

train_data.loc[(train_data['pickup_latitude'] == train_data['dropoff_latitude'])]

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2204,8.9,-73.936667,40.757815,0.0000,40.757815,1.0
6612,5.7,-74.008400,40.725900,-73.9947,40.725900,1.0


### Linear Regression

In [26]:
#Splitting data into x and y

y_train = train_data['fare_amount']

x_train = train_data.drop(['fare_amount'], axis=1 )

x_test = test_data.drop(['pickup_datetime'], axis = 1)

In [27]:
x_test.shape

(9914, 5)

In [28]:
#Creating Linear Regression Model

from sklearn.linear_model import LinearRegression


In [36]:
linear_model = LinearRegression().fit(x_train, y_train)

In [37]:
y_predict = linear_model.predict(x_test)

print(y_predict)

[16.12438523 16.12458593 16.12545766 ...  8.78783014  8.7904169
  8.77983387]


In [38]:
train_predict = linear_model.predict(x_train)

print(train_predict)

[16.13392055 16.13142296 14.6554225  ... 14.64880053 16.12727184
 16.12778135]


In [39]:
#Checking for mean squared error

from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_train, train_predict))
                              
print(rms)

437.8103783286747


In [34]:
print(y_train)

0         4.5
1        16.9
2         5.7
3         7.7
4         5.3
         ... 
15960    10.9
15961     6.5
15962    16.1
15963     8.5
15964     8.1
Name: fare_amount, Length: 15507, dtype: float64


In [40]:
test_predict = linear_model.predict(x_test)

print(test_predict)

[16.12438523 16.12458593 16.12545766 ...  8.78783014  8.7904169
  8.77983387]


### Decision Tree

In [42]:
from sklearn import tree

C50_model = tree.DecisionTreeRegressor().fit(x_train, y_train)

In [43]:
C50_Predictions = C50_model.predict(x_train)

In [44]:
C50_Predictions_test = C50_model.predict(x_test)
print(C50_Predictions_test)

[ 5.7  8.5  8.5 ... 45.  22.5  6.5]


In [45]:
print(C50_Predictions)

[ 4.5 16.9  5.7 ... 16.1  8.5  8.1]


In [46]:
print(y_train)

#Here, we can observe that the target variable fare_amount and C50_Predictions are same

0         4.5
1        16.9
2         5.7
3         7.7
4         5.3
         ... 
15960    10.9
15961     6.5
15962    16.1
15963     8.5
15964     8.1
Name: fare_amount, Length: 15507, dtype: float64


In [48]:
#Checking for mean squared error

rms_dt = sqrt(mean_squared_error(y_train, C50_Predictions))
                         
print(rms_dt)


#We got the least value in using Decision Tree model

3.6633104004497615e-16


In [59]:
#Checking for Mean Absolute Percentage Error

np.mean(np.abs((y_train - C50_Predictions) / y_train))*100

1.1085364267336346e-16

### Random Forest

In [49]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(n_estimators = 20).fit(x_train, y_train)

In [50]:
RF_Predictions = RF_model.predict(x_train)
print(RF_Predictions)

[ 5.535 18.01   6.24  ... 16.305  8.295  8.78 ]


In [51]:
rms_rf = sqrt(mean_squared_error(y_train, RF_Predictions))
                             
print(rms_rf)

200.4210551778837
